# Simulated Probit

In [12]:
import numpy as np
import gurobipy as grb
import scipy.sparse as spr

In [20]:
seed = 777
nbDraws = 1000
U_y = np.array([0.4, 0.5, 0.2, 0.3, 0.1, 0])
nbY = len(U_y)

In [21]:
rho = 0.5
Covar = rho * np.ones((nbY, nbY)) + (1 - rho) * np.eye(nbY)

In [22]:
E = np.linalg.eigh(Covar)
V = E[0]
Q = E[1]
SqrtCovar = Q.dot(np.diag(np.sqrt(V))).dot(Q.T)

In [23]:
epsilon_iy = np.random.normal(0,1,nbDraws*nbY).reshape(nbDraws,nbY).dot(SqrtCovar)
u_iy = epsilon_iy + U_y

In [24]:
ui = np.max(u_iy, axis=1)
s_y = np.sum((u_iy.T - ui).T == 0, axis=0) / nbDraws

In [26]:
A1 = spr.kron(np.ones((1, nbY)), spr.identity(nbDraws))
A2 = spr.kron(spr.identity(nbY), np.ones((1, nbDraws)))
A = spr.vstack([A1, A2])
obj = epsilon_iy.flatten(order='F')
rhs = np.ones(nbDraws)/nbDraws
rhs = np.append(rhs, s_y)
m = grb.Model('optimal')
x = m.addMVar(len(obj), name='couple')
m.setObjective(obj @ x, grb.GRB.MAXIMIZE)
m.addConstr(A @ x == rhs, name="Constr")

[<gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Update*>,
 <gurobi.Constr *Awaiting Model Up

In [27]:
m.optimize()
if m.Status == grb.GRB.Status.OPTIMAL:
    pi = m.getAttr('pi')
    Uhat_y = -np.subtract(pi[nbDraws:nbY+nbDraws], pi[nbY + nbDraws - 1])
    print('U_y (true and recovered)')
    print(U_y)
    print(Uhat_y)

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 1006 rows, 6000 columns and 12000 nonzeros
Model fingerprint: 0x60caf038
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-04, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-03, 3e-01]
Presolve time: 0.01s
Presolved: 1006 rows, 6000 columns, 12000 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3814338e+33   5.992000e+33   2.381434e+03      0s
    1246    8.5604359e-01   0.000000e+00   0.000000e+00      0s

Solved in 1246 iterations and 0.05 seconds
Optimal objective  8.560435898e-01
U_y (true and recovered)
[0.4 0.5 0.2 0.3 0.1 0. ]
[ 0.40183824  0.50078191  0.1961657   0.29671137  0.10307717 -0.        ]
